In [1]:
from ast import literal_eval
from os import listdir
from os.path import isfile, join
from scipy.sparse import csr_matrix, load_npz, save_npz
from tqdm import tqdm

import seaborn as sns
import datetime
import json
import numpy as np
import pandas as pd
import time
import yaml
import scipy.sparse as sparse
from ast import literal_eval

In [4]:
# Load Data
df_train = pd.read_csv('data/beer/Train.csv')
df_valid = pd.read_csv('data/beer/Valid.csv')
df_test = pd.read_csv('data/beer/Test.csv')
key_phrase = pd.read_csv('data/beer/KeyPhrases.csv')

In [5]:
# Load Data 2
rtrain = load_npz("data/beer/Rtrain.npz")
rvalid = load_npz("data/beer/Rvalid.npz")
rtest = load_npz("data/beer/Rtest.npz")

In [6]:
rtrain

<6370x3668 sparse matrix of type '<type 'numpy.float32'>'
	with 167511 stored elements in Compressed Sparse Row format>

In [8]:
df_train.head()

,beer/ABV,beer/name,beer/style,review/overall,review/text,review/timeUnix,user/profileName,Binary,UserIndex,ItemIndex,review,conca_review,keyVector,keyphrases_indices_length
0,5.5,Caldera Pale Ale,American Pale Ale (APA),4.0,Caldera- Pale Ale\t\tPicked up at Grape & Gour...,1251327677,NJpadreFan,0,1767,659,"['caldera', 'pale', 'ale', 'picked', 'grape', ...",caldera pale ale picked grape gourmet vb ligh...,"[1, 7, 12, 14, 16, 29, 31, 34, 39, 40, 42, 45]",12
1,5.5,Caldera Pale Ale,American Pale Ale (APA),2.5,12 oz. can poured into a pint glass. I really ...,1250928902,vacax,0,6159,659,"['oz', 'poured', 'pint', 'glass', 'really', 'l...",oz poured pint glass really like beer like pa...,"[1, 16, 18, 20, 24, 39, 42, 43, 45, 55]",10
2,5.5,Caldera Pale Ale,American Pale Ale (APA),4.0,A: Pours a cloudy orange with a crisp white tw...,1249866208,d0ggnate,0,3516,659,"['pours', 'cloudy', 'orange', 'crisp', 'white'...",pours cloudy orange crisp white two finger he...,"[12, 36, 39, 42]",4
3,5.5,Caldera Pale Ale,American Pale Ale (APA),4.5,Received this can through Beer Powerball. I am...,1249847121,babyhobbes,1,2893,659,"['received', 'beer', 'powerball', 'really', 's...",received beer powerball really sorry nt know ...,"[12, 42]",2
4,5.5,Caldera Pale Ale,American Pale Ale (APA),4.5,355ml can bought from Beersofeurope.com...\t\t...,1249556277,mdagnew,1,4858,659,"['bought', 'beersofeurope', 'com', 'poured', '...",bought beersofeurope com poured golden yellow...,"[1, 5, 7, 12, 14, 16, 18, 19, 26, 31, 34, 40, ...",18


In [9]:
# Models
from sklearn.metrics.pairwise import cosine_similarity
def train(matrix_train):
    similarity = cosine_similarity(X=matrix_train, Y=None, dense_output=True)
    return similarity

def get_I_K(df, row_name = 'ItemIndex', shape = (3668,75)):
    rows = []
    cols = []
    vals = []
    for i in tqdm(range(df.shape[0])):
        key_vector = literal_eval(df['keyVector'][i])
        rows.extend([df[row_name][i]]*len(key_vector)) ## Item index
        cols.extend(key_vector) ## Keyword Index
#         if binary:
        vals.extend(np.array([1]*len(key_vector)))
#         else:
#             vals.extend(arr[arr.nonzero()])    
    return csr_matrix((vals, (rows, cols)), shape=shape)


def predict(matrix_train, k, similarity, item_similarity_en = False):
    prediction_scores = []
    
    if item_similarity_en:
        matrix_train = matrix_train.transpose()
        
    for user_index in tqdm(range(matrix_train.shape[0])):
        # Get user u's prediction scores for all items
        vector_u = similarity[user_index]

        # Get closest K neighbors excluding user u self
        similar_users = vector_u.argsort()[::-1][1:k+1]
        # Get neighbors similarity weights and ratings
        similar_users_weights = similarity[user_index][similar_users]
        similar_users_ratings = matrix_train[similar_users].toarray()

        prediction_scores_u = similar_users_ratings * similar_users_weights[:, np.newaxis]

        prediction_scores.append(np.sum(prediction_scores_u, axis=0))
    res = np.array(prediction_scores)
    if item_similarity_en:
        res = res.transpose()
    return res

def prediction(prediction_score, topK, matrix_Train):

    prediction = []

    for user_index in tqdm(range(matrix_Train.shape[0])):
        vector_u = prediction_score[user_index]
        vector_train = matrix_Train[user_index]
        if len(vector_train.nonzero()[0]) > 0:
            vector_predict = sub_routine(vector_u, vector_train, topK=topK)
        else:
            vector_predict = np.zeros(topK, dtype=np.float32)

        prediction.append(vector_predict)

    return np.vstack(prediction)


def sub_routine(vector_u, vector_train, topK=500):

    train_index = vector_train.nonzero()[1]

    vector_u = vector_u

    candidate_index = np.argpartition(-vector_u, topK+len(train_index))[:topK+len(train_index)]
    vector_u = candidate_index[vector_u[candidate_index].argsort()[::-1]]
    vector_u = np.delete(vector_u, np.isin(vector_u, train_index).nonzero()[0])

    return vector_u[:topK]


In [10]:
u_similarity = train(rtrain)

In [13]:
U_K = get_I_K(df_train, row_name = 'UserIndex', shape = (6370,75))

100%|███████████████████████████████████████████████████████████████████████| 641773/641773 [00:32<00:00, 19864.83it/s]


In [14]:
res = predict(U_K, 10, u_similarity)

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:02<00:00, 2432.23it/s]


In [33]:
U_K[0][U_K[0].nonzero()]

matrix([[ 7, 35, 27,  1,  6,  5, 13,  6,  9,  4,  7, 13,  6, 44,  1, 20,
          4,  7,  2,  7, 16,  8,  1,  4, 11,  3,  1, 16, 14,  4,  3,  9,
          2, 19,  2,  3, 13, 10,  5,  6, 22, 17,  5, 11,  1,  1, 13,  4,
          1,  5,  4, 10,  4, 15,  7,  4, 12, 10,  7,  2,  1, 15,  2,  6,
          2,  2]])

In [36]:
res[0]

array([ 84.00923042, 215.8694369 , 105.07053825,   8.86858186,
        25.78474239,  51.06063677,  78.86394581,  21.29825376,
        63.25317055,  25.95691679,  28.74760556,  41.67915888,
        85.61797327,  16.12536277, 144.97887605,   3.25411236,
       143.26558426,  10.80280311,  27.22940387,  32.16788881,
        22.40749618,  68.35865185,  22.02200086,  17.6461527 ,
        30.71776073,   6.84443329,  52.27788997,  17.57675762,
        11.94877864,   5.81463091,  42.41261208,  36.93191479,
        29.80877489,  10.17466521,  41.2171516 ,  11.58094957,
        21.56327735,   3.67121929,  21.46959995,  85.16300964,
        37.56942199,  52.77996747, 101.33686914, 105.63312288,
       108.88178651,  98.73462543,  29.30211134,  20.35339764,
        19.46589303,   3.7303917 ,  76.21329193,  31.93960953,
        15.91736342,   4.90339676,   5.75619879,  69.36156599,
        18.50684226,   1.58225504,  59.92660914,  12.62514187,
        16.19339716,  27.11780505,  30.83943269,   6.19